# 1、大模型在金融行业有哪些应用场景？
- 金融文本分类：向模型解释什么叫作「文本分类任务」; 需要让模型按照我们指定的格式输出 使用：Incontext Learning
- 金融信息抽取：向模型解释什么叫作「信息抽取任务」; 让模型按照我们指定的格式（json）输出，使用技术Incontext Learning 
- 金融文本匹配：向模型解释什么叫作「文本匹配任务」; 需要让模型按照我们指定的格式输出


In [7]:
!conda install rich -y

Channels:
 - defaults
Platform: osx-64
Solving environment: done


==> WARNING: A newer version of conda exists. <==
    current version: 25.5.0
    latest version: 25.5.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /Users/BrownSugar/miniconda3/envs/AI-Lesson

  added / updated specs:
    - rich


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    markdown-it-py-2.2.0       |  py310hecd8cb5_1         115 KB
    mdurl-0.1.0                |  py310hecd8cb5_0          19 KB
    rich-13.9.4                |  py310hecd8cb5_0         511 KB
    ------------------------------------------------------------
                                           Total:         645 KB

The following NEW packages will be INSTALLED:

  markdown-it-py     pkgs/main/osx-64::markdown-it-py-2.2.0-py310hecd8cb5_1 
  mdurl              pkgs/

In [30]:
# 导入大模型平台（这里使用第三方集成平台替代）
from secret.thirdPartLLM import request_third_part_llm


# msg = [{"role": "user", "content": "周树人和鲁迅是兄弟吗？"}]
# msg = [{'role': 'system',
#         'content': "现在你是一个文本分类器，你需要按照要求将我给你的句子分类到：['新闻报道', '财务报告', '公司公告', '分析师报告']类别中。"},
#        {'role': 'user',
#         'content': "“今日，股市经历了一轮震荡，受到宏观经济数据和全球贸易紧张局势的影响。投资者密切关注美联储可能的政策调整，以适应市场的不确定性。”是 ['新闻报道', '财务报告', '公司公告', '分析师报告'] 里的什么类别？"},
#        {'role': 'assistant', 'content': '新闻报道'}, {'role': 'user',
#                                                       'content': "“本公司年度财务报告显示，去年公司实现了稳步增长的盈利，同时资产负债表呈现强劲的状况。经济环境的稳定和管理层的有效战略执行为公司的健康发展奠定了基础。”是 ['新闻报道', '财务报告', '公司公告', '分析师报告'] 里的什么类别？"},
#        {'role': 'assistant', 'content': '财务报告'}, {'role': 'user',
#                                                       'content': "“本公司高兴地宣布成功完成最新一轮并购交易，收购了一家在人工智能领域领先的公司。这一战略举措将有助于扩大我们的业务领域，提高市场竞争力”是 ['新闻报道', '财务报告', '公司公告', '分析师报告'] 里的什么类别？"},
#        {'role': 'assistant', 'content': '公司公告'}, {'role': 'user',
#                                                       'content': "“最新的行业分析报告指出，科技公司的创新将成为未来增长的主要推动力。云计算、人工智能和数字化转型被认为是引领行业发展的关键因素，投资者应关注这些趋势”是 ['新闻报道', '财务报告', '公司公告', '分析师报告'] 里的什么类别？"},
#        {'role': 'assistant', 'content': '分析师报告'}, {'role': 'user',
#                                                         'content': "“本公司宣布成功收购一家在创新科技领域领先的公司，这一战略性收购将有助于公司拓展技术能力和加速产品研发。”是 ['新闻报道', '财务报告', '公司公告', '分析师报告'] 里的什么类别？"}
# ]

def clean_response(response: str):
    """
    后处理模型输出。

    Args:
        response (str): _description_
    """
    # response1='```json["name":lucy]```abc```json["name":lucy]```'
    if '```json' in response:
        res = re.findall(r'```json(.*?)```', response, re.DOTALL)
        if len(res) and res[0]:
            response = res[0]
        response.replace('、', ',')
    try:
        return json.loads(response)
    except:
        return response


msg = [{'role': 'system', 'content': '你是一个信息抽取助手。'}, {'role': 'user',
                                                                'content': "2023-01-10，股市震荡。股票古哥-D[EOOE]美股今日开盘价100美元，一度飙升至105美元，随后回落至98美元，最终以102美元收盘，成交量达到520000。\n\n提取上述句子中“金融”(日期, 股票名称, 开盘价, 收盘价, 成交量)的实体，并按照JSON格式输出，上述句子中不存在的信息用['原文中未提及']来表示，多个值之间用','分隔。"},
       {'role': 'assistant',
        'content': '{"日期": ["2023-01-10"], "股票名称": ["古哥-D[EOOE]美股"], "开盘价": ["100美元"], "收盘价": ["102美元"], "成交量": ["520000"]}'},
       {'role': 'user',
        'content': "2023-02-15，寓意吉祥的节日，股票佰笃[BD]美股开盘价10美元，虽然经历了波动，但最终以13美元收盘，成交量微幅增加至460,000，投资者情绪较为平稳。\n\n提取上述句子中“金融”(日期, 股票名称, 开盘价, 收盘价, 成交量)的实体，并按照JSON格式输出，上述句子中不存在的信息用['原文中未提及']来表示，多个值之间用','分隔。"}]

result = request_third_part_llm('qwen2.5-72b-instruct', msg)
print(result)
print('\n')
temp = result['choices'][0]['message']['content']
print(temp)
print('\n')
print(clean_response(temp))

{'choices': [{'message': {'role': 'assistant', 'content': '{"日期": ["2023-02-15"], "股票名称": ["佰笃[BD]美股"], "开盘价": ["10美元"], "收盘价": ["13美元"], "成交量": ["460,000"]}'}, 'finish_reason': 'stop', 'index': 0, 'logprobs': None}], 'object': 'chat.completion', 'usage': {'prompt_tokens': 331, 'completion_tokens': 57, 'total_tokens': 388}, 'created': 1749380037, 'system_fingerprint': None, 'model': 'qwen2.5-72b-instruct', 'id': 'chatcmpl-be4a2878-beb9-9fdf-956a-1239f8c423e5'}


{"日期": ["2023-02-15"], "股票名称": ["佰笃[BD]美股"], "开盘价": ["10美元"], "收盘价": ["13美元"], "成交量": ["460,000"]}


{'日期': ['2023-02-15'], '股票名称': ['佰笃[BD]美股'], '开盘价': ['10美元'], '收盘价': ['13美元'], '成交量': ['460,000']}


In [19]:
msg = {'choices': [
    {'message': {'role': 'assistant', 'content': '公司公告'}, 'finish_reason': 'stop', 'index': 0, 'logprobs': None}],
    'object': 'chat.completion', 'usage': {'prompt_tokens': 373, 'completion_tokens': 2, 'total_tokens': 375},
    'created': 1749378476, 'system_fingerprint': None, 'model': 'qwen2.5-72b-instruct',
    'id': 'chatcmpl-ac6c9cc3-106d-9ad0-bbd9-f99738762f1f'}

print(msg['choices'])
print(msg['choices'][0]['message'])
print(msg['message']['content'])

[{'message': {'role': 'assistant', 'content': '公司公告'}, 'finish_reason': 'stop', 'index': 0, 'logprobs': None}]
{'role': 'assistant', 'content': '公司公告'}


KeyError: 'message'

In [22]:
import time
# 导入rich模块
# from rich import print
# 导入富文本控制台
# from rich.console import Console
# 导入大模型平台（这里使用第三方集成平台替代）
from secret.thirdPartLLM import request_third_part_llm

class_examples = {
    '新闻报道': '今日，股市经历了一轮震荡，受到宏观经济数据和全球贸易紧张局势的影响。投资者密切关注美联储可能的政策调整，以适应市场的不确定性。',
    '财务报告': '本公司年度财务报告显示，去年公司实现了稳步增长的盈利，同时资产负债表呈现强劲的状况。经济环境的稳定和管理层的有效战略执行为公司的健康发展奠定了基础。',
    '公司公告': '本公司高兴地宣布成功完成最新一轮并购交易，收购了一家在人工智能领域领先的公司。这一战略举措将有助于扩大我们的业务领域，提高市场竞争力',
    '分析师报告': '最新的行业分析报告指出，科技公司的创新将成为未来增长的主要推动力。云计算、人工智能和数字化转型被认为是引领行业发展的关键因素，投资者应关注这些趋势'
}


def init_prompts():
    '''
    初始化前置的prompt，便于模型做in context learning
    :return: 
    '''
    class_list = list(class_examples.keys())
    # print(class_list)

    pre_history = [{"role": "system",
                    "content": f"现在你是一个文本分类器，你需要按照要求将我给你的句子分类到：{class_list}类别中。"}, ]
    for _type, example in class_examples.items():
        pre_history.append({"role": "user", "content": f'“{example}”是 {class_list} 里的什么类别？'})
        pre_history.append({"role": "assistant", "content": _type})

    return {'class_list': class_list, 'pre_history': pre_history}


def inference(sentences: list, custom_settings: dict):
    for sentence in sentences:
        sentence_with_prompt = f"“{sentence}”是 {custom_settings['class_list']} 里的什么类别？"
        msg = [*custom_settings['pre_history'], {"role": 'user', "content": sentence_with_prompt}]
        print(msg)
        print('\n')
        response = request_third_part_llm('qwen2.5-72b-instruct', msg)
        print(response)
        print('\n')
        response = response['choices'][0]['message']['content']
        print('输出结果：' + response)
        print('\n')
        time.sleep(10)


if __name__ == '__main__':
    # console = Console()

    sentences = [
        "今日，央行发布公告宣布降低利率，以刺激经济增长。这一降息举措将影响贷款利率，并在未来几个季度内对金融市场产生影响。",
        "本公司宣布成功收购一家在创新科技领域领先的公司，这一战略性收购将有助于公司拓展技术能力和加速产品研发。",
        "公司资产负债表显示，公司偿债能力强劲，现金流充足，为未来投资和扩张提供了坚实的财务基础。",
        "最新的分析报告指出，可再生能源行业预计将在未来几年经历持续增长，投资者应该关注这一领域的投资机会",
    ]

    custom_settings = init_prompts()
    # print(custom_settings)
    inference(sentences, custom_settings)


[{'role': 'system', 'content': "现在你是一个文本分类器，你需要按照要求将我给你的句子分类到：['新闻报道', '财务报告', '公司公告', '分析师报告']类别中。"}, {'role': 'user', 'content': "“今日，股市经历了一轮震荡，受到宏观经济数据和全球贸易紧张局势的影响。投资者密切关注美联储可能的政策调整，以适应市场的不确定性。”是 ['新闻报道', '财务报告', '公司公告', '分析师报告'] 里的什么类别？"}, {'role': 'assistant', 'content': '新闻报道'}, {'role': 'user', 'content': "“本公司年度财务报告显示，去年公司实现了稳步增长的盈利，同时资产负债表呈现强劲的状况。经济环境的稳定和管理层的有效战略执行为公司的健康发展奠定了基础。”是 ['新闻报道', '财务报告', '公司公告', '分析师报告'] 里的什么类别？"}, {'role': 'assistant', 'content': '财务报告'}, {'role': 'user', 'content': "“本公司高兴地宣布成功完成最新一轮并购交易，收购了一家在人工智能领域领先的公司。这一战略举措将有助于扩大我们的业务领域，提高市场竞争力”是 ['新闻报道', '财务报告', '公司公告', '分析师报告'] 里的什么类别？"}, {'role': 'assistant', 'content': '公司公告'}, {'role': 'user', 'content': "“最新的行业分析报告指出，科技公司的创新将成为未来增长的主要推动力。云计算、人工智能和数字化转型被认为是引领行业发展的关键因素，投资者应关注这些趋势”是 ['新闻报道', '财务报告', '公司公告', '分析师报告'] 里的什么类别？"}, {'role': 'assistant', 'content': '分析师报告'}, {'role': 'user', 'content': "“今日，央行发布公告宣布降低利率，以刺激经济增长。这一降息举措将影响贷款利率，并在未来几个季度内对金融市场产生影响。”是 ['新闻报道', '财务报告', '公司公告', '分析师报告'] 里的什么类别？"}]


{'c

In [31]:
import json
import re

# 定义不同实体下的具备属性
schema = {'金融': ['日期', '股票名称', '开盘价', '收盘价', '成交量'], }

IE_PATTERN = "{}\n\n提取上述句子中{}的实体，并按照JSON格式输出，上述句子中不存在的信息用['原文中未提及']来表示，多个值之间用','分隔。"

# 提供一些例子供模型参考
ie_examples = {
    '金融': [
        {
            'content': '2023-01-10，股市震荡。股票古哥-D[EOOE]美股今日开盘价100美元，一度飙升至105美元，随后回落至98美元，最终以102美元收盘，成交量达到520000。',
            'answers': {
                '日期': ['2023-01-10'],
                '股票名称': ['古哥-D[EOOE]美股'],
                '开盘价': ['100美元'],
                '收盘价': ['102美元'],
                '成交量': ['520000'],
            }
        }
    ]
}


def init_prompts():
    ie_pre_history = [{"role": "system", "content": "你是一个信息抽取助手。"}, ]
    for _type, example_list in ie_examples.items():
        for example in example_list:
            sentence = example['content']
            properties_str = ', '.join(schema[_type])
            schema_str_list = f'“{_type}”({properties_str})'

            sentence_with_prompt = IE_PATTERN.format(sentence, schema_str_list)

            ie_pre_history.append({"role": "user", "content": f'{sentence_with_prompt}'})
            ie_pre_history.append(
                {"role": "assistant", "content": f"{json.dumps(example['answers'], ensure_ascii=False)}"})

    return {'ie_pre_history': ie_pre_history}


def clean_response(response: str):
    ...


def inference(sentences: list, custom_settings: dict):
    for sentence in sentences:
        cls_res = "金融"
        if cls_res not in schema:
            print(f'The type model inferenced {cls_res} which is not in schema dict, exited.')
            exit()

        properties_str = ', '.join(schema[cls_res])
        schema_str_list = f'“{cls_res}”({properties_str})'
        sentence_with_ie_prompt = IE_PATTERN.format(sentence, schema_str_list)

        msg = [*custom_settings['ie_pre_history'], {"role": "user", "content": sentence_with_ie_prompt}]
        print(msg)
        print('\n')
        response = request_third_part_llm('qwen2.5-72b-instruct', msg)
        print(response)
        print('\n')
        response = response['choices'][0]['message']['content']
        print('输出结果：' + response)
        time.sleep(5)


if __name__ == '__main__':
    # 初始化句子和自定义设置
    sentences = [
        '2023-02-15，寓意吉祥的节日，股票佰笃[BD]美股开盘价10美元，虽然经历了波动，但最终以13美元收盘，成交量微幅增加至460,000，投资者情绪较为平稳。',
        '2023-04-05，市场迎来轻松氛围，股票盘古(0021)开盘价23元，尽管经历了波动，但最终以26美元收盘，成交量缩小至310,000，投资者保持观望态度。',
    ]

    # 初始化自定义设置
    custom_settings = init_prompts()

    # 开始推理
    inference(
        sentences,
        custom_settings
    )


[{'role': 'system', 'content': '你是一个信息抽取助手。'}, {'role': 'user', 'content': "2023-01-10，股市震荡。股票古哥-D[EOOE]美股今日开盘价100美元，一度飙升至105美元，随后回落至98美元，最终以102美元收盘，成交量达到520000。\n\n提取上述句子中“金融”(日期, 股票名称, 开盘价, 收盘价, 成交量)的实体，并按照JSON格式输出，上述句子中不存在的信息用['原文中未提及']来表示，多个值之间用','分隔。"}, {'role': 'assistant', 'content': '{"日期": ["2023-01-10"], "股票名称": ["古哥-D[EOOE]美股"], "开盘价": ["100美元"], "收盘价": ["102美元"], "成交量": ["520000"]}'}, {'role': 'user', 'content': "2023-02-15，寓意吉祥的节日，股票佰笃[BD]美股开盘价10美元，虽然经历了波动，但最终以13美元收盘，成交量微幅增加至460,000，投资者情绪较为平稳。\n\n提取上述句子中“金融”(日期, 股票名称, 开盘价, 收盘价, 成交量)的实体，并按照JSON格式输出，上述句子中不存在的信息用['原文中未提及']来表示，多个值之间用','分隔。"}]


{'choices': [{'message': {'role': 'assistant', 'content': '{\n  "日期": ["2023-02-15"],\n  "股票名称": ["佰笃[BD]美股"],\n  "开盘价": ["10美元"],\n  "收盘价": ["13美元"],\n  "成交量": ["460,000"]\n}'}, 'finish_reason': 'stop', 'index': 0, 'logprobs': None}], 'object': 'chat.completion', 'usage': {'prompt_tokens': 331, 'completion_tokens': 64, 'total_tokens': 395}, 'created': 1749380251, 'system_fingerprint': 

In [33]:

# 提供相似，不相似的语义匹配例子
examples = {
    '是': [
        ('公司ABC发布了季度财报，显示盈利增长。', '财报披露，公司ABC利润上升。'),
    ],
    '不是': [
        ('黄金价格下跌，投资者抛售。', '外汇市场交易额创下新高。'),
        ('央行降息，刺激经济增长。', '新能源技术的创新。')
    ]
}


def init_prompts():
    """
    初始化前置prompt，便于模型做 incontext learning。
    """
    pre_history = [{"role": "system",
                    "content": "现在你需要帮助我完成文本匹配任务，当我给你两个句子时，你需要回答我这两句话语义是否相似。只需要回答是否相似，不要做多余的回答。"}, ]
    for key, sentence_pairs in examples.items():
        for sentence_pair in sentence_pairs:
            sentence1, sentence2 = sentence_pair
            pre_history.append(
                {"role": "user", "content": f'句子一: {sentence1}\n句子二: {sentence2}\n上面两句话是相似的语义吗？'})
            pre_history.append({"role": "assistant", "content": key})

    return {'pre_history': pre_history}


def inference(sentence_pairs: list, custom_settings: dict):
    for sentence_pair in sentence_pairs:
        sentence1, sentence2 = sentence_pair
        sentence_with_prompt = f'句子一: {sentence1}\n句子二: {sentence2}\n上面两句话是相似的语义吗？'

        msg = [*custom_settings["pre_history"], {"role": 'user', "content": sentence_with_prompt}]
        print(msg)
        print('\n')
        # response = ollama.chat(model="qwen2.5:7b", messages=msg)
        response = request_third_part_llm('qwen2.5-72b-instruct', msg)
        print(response)
        print('\n')
        response = response['choices'][0]['message']['content']
        print('输出结果：' + response)
        time.sleep(5)


if __name__ == '__main__':
    sentence_pairs = [
        ('股票市场今日大涨，投资者乐观。', '持续上涨的市场让投资者感到满意。'),
        ('油价大幅下跌，能源公司面临挑战。', '未来智能城市的建设趋势愈发明显。'),
        ('利率上升，影响房地产市场。', '高利率对房地产有一定冲击。'),
    ]

    custom_settings = init_prompts()
    inference(
        sentence_pairs,
        custom_settings
    )

[{'role': 'system', 'content': '现在你需要帮助我完成文本匹配任务，当我给你两个句子时，你需要回答我这两句话语义是否相似。只需要回答是否相似，不要做多余的回答。'}, {'role': 'user', 'content': '句子一: 公司ABC发布了季度财报，显示盈利增长。\n句子二: 财报披露，公司ABC利润上升。\n上面两句话是相似的语义吗？'}, {'role': 'assistant', 'content': '是'}, {'role': 'user', 'content': '句子一: 黄金价格下跌，投资者抛售。\n句子二: 外汇市场交易额创下新高。\n上面两句话是相似的语义吗？'}, {'role': 'assistant', 'content': '不是'}, {'role': 'user', 'content': '句子一: 央行降息，刺激经济增长。\n句子二: 新能源技术的创新。\n上面两句话是相似的语义吗？'}, {'role': 'assistant', 'content': '不是'}, {'role': 'user', 'content': '句子一: 股票市场今日大涨，投资者乐观。\n句子二: 持续上涨的市场让投资者感到满意。\n上面两句话是相似的语义吗？'}]


{'choices': [{'message': {'role': 'assistant', 'content': '是'}, 'finish_reason': 'stop', 'index': 0, 'logprobs': None}], 'object': 'chat.completion', 'usage': {'prompt_tokens': 226, 'completion_tokens': 1, 'total_tokens': 227}, 'created': 1749380476, 'system_fingerprint': None, 'model': 'qwen2.5-72b-instruct', 'id': 'chatcmpl-fd8e9183-8aa1-9468-b08e-36af23fb2845'}


输出结果：是
[{'role': 'system', 'content': '现在你需要帮助我完成文本匹配任务，当我给